SPARK INTRODUCTION EXERCICE 1

In [0]:
%python
df_s_original = spark.read.text("/Volumes/workspace/default/tp1")

In [0]:
df_s_original.show(n=1000, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                     |
+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|word count from Wikipedia the free encyclopedia                                                                                                           |
|the word count is the number of words in a document or passage of text Word counting may be needed when a text                                            |
|is required to stay within certain numbers of words This may particularly be the case in academia legal                                                   |
|proceedings journalism and advertising Word count is comm

In [0]:
%sql
-- Creer le volume(emplacement) pour le fichier, si il n existe pas deja
CREATE VOLUME IF NOT EXISTS default.tp1_output1;

In [0]:
df_s_original.write.mode("overwrite").text("/Volumes/workspace/default/tp1_output1")

In [0]:
%python
df_s = spark.read.text("/Volumes/workspace/default/tp1_output1")

In [0]:
# Connaître le nombre de blocs
df_s.count()

135

In [0]:
# Donner le premier element
df_s.first()

Row(value='word count from Wikipedia the free encyclopedia')

In [0]:
# Donner les 10 premiers elements
df_s.head(10)

[Row(value='word count from Wikipedia the free encyclopedia'),
 Row(value='the word count is the number of words in a document or passage of text Word counting may be needed when a text'),
 Row(value='is required to stay within certain numbers of words This may particularly be the case in academia legal'),
 Row(value='proceedings journalism and advertising Word count is commonly used by translators to determine the price for'),
 Row(value='the translation job Word counts may also be used to calculate measures of readability and to measure typing'),
 Row(value='and reading speeds usually in words per minute When converting character counts to words a measure of five or'),
 Row(value='six characters to a word is generally used Contents Details and variations of definition Software In fiction'),
 Row(value='In non fiction See also References Sources External links Details and variations of definition'),
 Row(value='This section does not cite any references or sources Please help improve t

In [0]:
# Donner 10 premiers elements sous forme de liste
elements_list = [row.value for row in df_s.take(10)]
print(elements_list)

['word count from Wikipedia the free encyclopedia', 'the word count is the number of words in a document or passage of text Word counting may be needed when a text', 'is required to stay within certain numbers of words This may particularly be the case in academia legal', 'proceedings journalism and advertising Word count is commonly used by translators to determine the price for', 'the translation job Word counts may also be used to calculate measures of readability and to measure typing', 'and reading speeds usually in words per minute When converting character counts to words a measure of five or', 'six characters to a word is generally used Contents Details and variations of definition Software In fiction', 'In non fiction See also References Sources External links Details and variations of definition', 'This section does not cite any references or sources Please help improve this section by adding citations to', 'reliable sources Unsourced material may be challenged and removed']


SPARK INTRODUCTION EXERCICE 2

In [0]:
%sql
-- Creer le volume(emplacement) pour le fichier, si il n existe pas deja
CREATE VOLUME IF NOT EXISTS default.Filter_word;

In [0]:
# Créer un fichier contenant uniquement les blocs ayant le mot "words"
df_words_filtered = df_s.filter(df_s.value.contains("word"))
print(df_words_filtered)
df_words_filtered.write.mode("overwrite").text("/Volumes/workspace/default/Filter_word")

DataFrame[value: string]


In [0]:
# 10 premiers items contenant le mot "words"
df_words_filtered.take(10)

[Row(value='word count from Wikipedia the free encyclopedia'),
 Row(value='the word count is the number of words in a document or passage of text Word counting may be needed when a text'),
 Row(value='is required to stay within certain numbers of words This may particularly be the case in academia legal'),
 Row(value='and reading speeds usually in words per minute When converting character counts to words a measure of five or'),
 Row(value='six characters to a word is generally used Contents Details and variations of definition Software In fiction'),
 Row(value='Variations in the operational definitions of how to count the words can occur namely what counts as a word and'),
 Row(value="which words don't count toward the total However especially since the advent of widespread word processing there"),
 Row(value='The consensus is to accept the text segmentation rules generally found in most word processing software including how'),
 Row(value='word boundaries are determined which depends

In [0]:
# compter le nombre de lettres par blocs
print([len(row.value) for row in df_words_filtered.take(10)])

[47, 110, 103, 109, 108, 110, 112, 117, 154, 123]


In [0]:
### compter le nombre de lettre du fichier
from pyspark.sql.functions import length

# Ajouter une colonne "length" contenant le nombre de lettres dans chaque ligne
df_with_length = df_words_filtered.withColumn("length", length(df_words_filtered.value))

# Calculer le nombre total de lettres en sommant la colonne "length"
total_letters = df_with_length.agg({"length": "sum"}).collect()[0][0]

# Afficher le résultat
print(f"Nombre total de lettres dans le fichier : {total_letters}")


Nombre total de lettres dans le fichier : 9759


In [0]:
# Séparer chaque ligne en mots et les aplatir (flatMap)
from pyspark.sql.functions import split, explode

mots = df_words_filtered.select(explode(split(df_words_filtered["value"], " ")).alias("mot"))
# Afficher les mots
mots.show(10, truncate=False)

+------------+
|mot         |
+------------+
|word        |
|count       |
|from        |
|Wikipedia   |
|the         |
|free        |
|encyclopedia|
|the         |
|word        |
|count       |
+------------+
only showing top 10 rows


In [0]:
#### Utiliser MapinPandas car Map n'est pas accessible (pas de rdd sur databrick)
# Impossible d'afficher les resultat car problème de conflit de version python lors de l'affichage
# bref, spark et databricks c est pas ouf ...

import pandas as pd
# Fonction de transformation : division de la ligne en mots
def split_into_words(p_df: pd.DataFrame) -> pd.DataFrame:
    # Divise la colonne 'value' en mots (en supposant que 'value' est une chaîne de caractères)
    p_df['words'] = p_df['value'].apply(lambda x: x.split())  # Split sur les espaces
    return p_df[['words']]  # Retourne seulement la colonne 'words'

# Appliquer la transformation avec mapInPandas
df_words = df_words_filtered.mapInPandas(split_into_words, schema="words array<string>")

# Afficher le résultat
#df_words.show(truncate=False)

##### a. Quelle est la différence entre ces deux fonctions ?
Impossible d'utiliser FlatMap et Map car RDD ne fonctionne pas sur Databrick.
On a donc fait le FlatMap en .select(explode()
et le Map en .mapInPandas
##### b. Le résultat retourné doit être stocké dans la mémoire. Quelle fonction utiliser ?
rdd.cache()
Cela ne marche pas sur Datrabrick car RDD n'est pas disponible